# Descriptive Statistics Automation Project

The project aims to automate whatever is possible within my knowledge, program tools to assist in data processing and analysis, practice knowledge and seek more.

Click run all and go to the last cell to download the descriptive analysis PDF

# Environment preparation

instala bibliotecas

chama as bibliotecas

prepara o pdf

faz a leitura dos dados (csv, xls, json)

faz a classificação dos tipos de dados

faz a limpeza dos dados

ativa o conjunto de operações de cada tipo, qualitativo, quantitativo

reune tudo em um pdf e salva com download


## Install libraries

In [1]:
%pip install weasyprint
%pip install -U kaleido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.9/297.9 kB 5.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 36.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 60.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 850.6/850.6 kB 26.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 18.9 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


## Import libraries

In [2]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import seaborn as sns
import plotly as plt
import pandas as pd
import numpy as np
import warnings
import base64
import io

from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
from sklearn.preprocessing import LabelEncoder
from scipy.stats import spearmanr, pearsonr
from scipy.stats import gaussian_kde
from weasyprint import HTML
from PIL import Image

In [3]:
warnings.filterwarnings('ignore')

## Prepare PDF

In [4]:
class PDFGenerator:
    def __init__(self):
        self.html_content = ""
        self.image_count = 0
        self.level = 1
    
    def add_header(self, text):
        self.html_content += f"<h{self.level}>{text}</h{self.level}>"
    
    def add_text(self, text):
        self.html_content += f"<p>{text}</p>"

    def add_break(self):
        self.html_content += f"<br />"
    
    def add_dataframe(self, df, style=False):
        if style:
            self.html_content += df.style.set_table_styles([
                {'selector': 'th', 'props': [('background-color', '#40466e'), ('color', 'white')],
                 'selector': 'td', 'props': [('border', '1px solid #ddd')]}
            ]).hide(axis='index').to_html()
        else:
            self.html_content += df.to_html()
    
    def add_image(self, image_path, max_width=600, max_height=800, quality=100, spacing_before=10, spacing_after=10, margin_right=10, margin_left=10):
        """
        Adiciona imagem com espaçamento controlado
        
        Parâmetros adicionais:
        - spacing_before: Espaço antes da imagem (em pixels)
        - spacing_after: Espaço após a imagem (em pixels)
        """
        self.image_count += 1
        
        with Image.open(image_path) as img:
            width, height = img.size
            ratio = min(max_width/width, max_height/height)
            new_size = (int(width*ratio), int(height*ratio))
            img = img.resize(new_size, Image.Resampling.LANCZOS)
            
            img_bytes = io.BytesIO()
            img.save(img_bytes, format='PNG', quality=quality)
            img_bytes.seek(0)
            img_data = base64.b64encode(img_bytes.read()).decode('utf-8')
        
        # Adiciona espaçamento antes e depois da imagem
        self.html_content += (
            f'<div style="margin-top: {spacing_before}px {margin_right}px {spacing_after}px {margin_left}px; text-align:center;">'
            f'<img src="data:image/png;base64,{img_data}" '
            f'style="max-width:{max_width}px; max-height:{max_height}px; width:auto; height:auto;"/>'
            f'</div>'
        )
    
    def generate_pdf(self, filename="report.pdf"):
        # Usando WeasyPrint para melhor qualidade (instale com pip install weasyprint)
        HTML(string=self.html_content).write_pdf(filename)
        return filename

# Instancie o gerador
pdf_gen = PDFGenerator()


# Célula com texto
# pdf_gen.add_header("Análise Exploratória de Dados")
# pdf_gen.add_text("Esta é uma análise completa dos dados de vendas do trimestre.")

# Célula com DataFrame
# pdf_gen.add_dataframe(df, style=True)

# Célula com visualização
# import matplotlib.pyplot as plt
# plt.bar(df['Mês'], df['Vendas'])
# plt.title('Vendas por Mês')
# plt.savefig('vendas.png')
# plt.close()
# pdf_gen.add_image('vendas.png')

## Read Data

In [5]:
df = pd.read_csv('/kaggle/input/student-depression-dataset/student_depression_dataset.csv') #kaggle/input/folder/archive.csv

# .read_excel('arquivo.xlsx')
# .read_json('arquivo.json')

In [6]:
df.iloc[:, 1:]

,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,Male,33.0,Visakhapatnam,Student,5.0,0.0,8.97,2.0,0.0,'5-6 hours',Healthy,B.Pharm,Yes,3.0,1.0,No,1
1,Female,24.0,Bangalore,Student,2.0,0.0,5.90,5.0,0.0,'5-6 hours',Moderate,BSc,No,3.0,2.0,Yes,0
2,Male,31.0,Srinagar,Student,3.0,0.0,7.03,5.0,0.0,'Less than 5 hours',Healthy,BA,No,9.0,1.0,Yes,0
3,Female,28.0,Varanasi,Student,3.0,0.0,5.59,2.0,0.0,'7-8 hours',Moderate,BCA,Yes,4.0,5.0,Yes,1
4,Female,25.0,Jaipur,Student,4.0,0.0,8.13,3.0,0.0,'5-6 hours',Moderate,M.Tech,Yes,1.0,1.0,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27896,Female,27.0,Surat,Student,5.0,0.0,5.75,5.0,0.0,'5-6 hours',Unhealthy,'Class 12',Yes,7.0,1.0,Yes,0
27897,Male,27.0,Ludhiana,Student,2.0,0.0,9.40,3.0,0.0,'Less than 5 hours',Healthy,MSc,No,0.0,3.0,Yes,0
27898,Male,31.0,Faridabad,Student,3.0,0.0,6.61,4.0,0.0,'5-6 hours',Unhealthy,MD,No,12.0,2.0,No,0
27899,Female,18.0,Ludhiana,Student,5.0,0.0,6.88,2.0,0.0,'Less than 5 hours',Healthy,'Class 12',Yes,10.0,5.0,No,1


## Classification Data

In [7]:
pdf_gen.add_header("Visualização e Conferência do Dataset")

pdf_gen.add_dataframe(df[0:6], style=True)

df.head()

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,2,Male,33.0,Visakhapatnam,Student,5.0,0.0,8.97,2.0,0.0,'5-6 hours',Healthy,B.Pharm,Yes,3.0,1.0,No,1
1,8,Female,24.0,Bangalore,Student,2.0,0.0,5.90,5.0,0.0,'5-6 hours',Moderate,BSc,No,3.0,2.0,Yes,0
2,26,Male,31.0,Srinagar,Student,3.0,0.0,7.03,5.0,0.0,'Less than 5 hours',Healthy,BA,No,9.0,1.0,Yes,0
3,30,Female,28.0,Varanasi,Student,3.0,0.0,5.59,2.0,0.0,'7-8 hours',Moderate,BCA,Yes,4.0,5.0,Yes,1
4,32,Female,25.0,Jaipur,Student,4.0,0.0,8.13,3.0,0.0,'5-6 hours',Moderate,M.Tech,Yes,1.0,1.0,No,0


In [8]:
pdf_gen.add_text(f"""Tipo inicial dos dados: \n 
{df.dtypes}
\n""".replace("\n", "<br />"))

print(df.dtypes)

id                                         int64
Gender                                    object
Age                                      float64
City                                      object
Profession                                object
Academic Pressure                        float64
Work Pressure                            float64
CGPA                                     float64
Study Satisfaction                       float64
Job Satisfaction                         float64
Sleep Duration                            object
Dietary Habits                            object
Degree                                    object
Have you ever had suicidal thoughts ?     object
Work/Study Hours                         float64
Financial Stress                          object
Family History of Mental Illness          object
Depression                                 int64
dtype: object


Temos 4 tipos de dados

Qualitativos Nominais e Ordinais - string

Quantitativos Discretos e Continuos - int e float

e também o tipo especial data

In [9]:
# limpeza de símbolos monetários

'''df['Amount'] = df['Amount'].str.extract(r'(\d+[\.,]?\d*)')[0] \
                               .str.replace(',', '.') \
                               .astype(float)'''

"df['Amount'] = df['Amount'].str.extract(r'(\\d+[\\.,]?\\d*)')[0]                                .str.replace(',', '.')                                .astype(float)"

In [10]:
# transformação em data

'''df['Date'] = pd.to_datetime(df['Date'])'''

"df['Date'] = pd.to_datetime(df['Date'])"

In [11]:
df = df.convert_dtypes(convert_string=True, convert_floating=False, convert_integer=False)

In [12]:
pdf_gen.add_dataframe(df[0:3], style=True)

In [13]:
pdf_gen.add_text(f'''Verificando se, após a transformação os tipos dos dados correspondem aos da tabela: \n
{df.dtypes}
\n'''.replace("\n", "<br />"))


print(f'''Verificando se, após a transformação os tipos dos dados correspondem aos da tabela: \n

{df.dtypes}''')

Verificando se, após a transformação os tipos dos dados correspondem aos da tabela: 


id                                                int64
Gender                                   string[python]
Age                                             float64
City                                     string[python]
Profession                               string[python]
Academic Pressure                               float64
Work Pressure                                   float64
CGPA                                            float64
Study Satisfaction                              float64
Job Satisfaction                                float64
Sleep Duration                           string[python]
Dietary Habits                           string[python]
Degree                                   string[python]
Have you ever had suicidal thoughts ?    string[python]
Work/Study Hours                                float64
Financial Stress                         string[python]
Family History of

## Cleaning Data

In [14]:
linha = df.shape[0]

coluna = df.shape[1]

df.dropna(inplace=True)

linha1 = df.shape[0]

coluna1 = df.shape[1]


pdf_gen.add_text(f'Antes da limpeza de dados faltantes, o data set tinha {linha} linhas e {coluna} colunas. A limpeza retirou {linha - linha1} linhas e {coluna - coluna1} colunas, ficando o dataset atualmente com {linha1} linhas e {coluna1} colunas.\n'.replace("\n", "<br />"))

print(f'Antes da limpeza de dados faltantes, o data set tinha {linha} linhas e {coluna} colunas. A limpeza retirou {linha - linha1} linhas e {coluna - coluna1} colunas, ficando o dataset atualmente com {linha1} linhas e {coluna1} colunas.')

Antes da limpeza de dados faltantes, o data set tinha 27901 linhas e 18 colunas. A limpeza retirou 0 linhas e 0 colunas, ficando o dataset atualmente com 27901 linhas e 18 colunas.


In [15]:
for coluna in df.columns:
    if (df[coluna].dtype == 'int') or (df[coluna].dtype == 'float'):
        Q1 = df[coluna].quantile(0.25)
        Q3 = df[coluna].quantile(0.75)
        IQR = Q3 - Q1
        limite_inferior = Q1 - 1.5 * IQR
        limite_superior = Q3 + 1.5 * IQR

        contagem_inferiores = (df[coluna] < limite_inferior).sum()
        contagem_superiores = (df[coluna] > limite_superior).sum()

        df[coluna] = df[coluna].apply(
            lambda x: limite_inferior if x < limite_inferior else 
                     (limite_superior if x > limite_superior else x))

pdf_gen.add_text(f"A quantidade de outliers detectados nas variáveis quantitativas foram {contagem_inferiores + contagem_superiores}.\nCom {contagem_inferiores} valores abaixo do limite inferior e {contagem_superiores} valores acima do limite superior, e todos eles foram substituidos pelos seus respectivos limites em cada coluna de dados quantitativos. \n".replace("\n", "<br />"))

print(f"A quantidade de outliers detectados nas variáveis quantitativas foram {contagem_inferiores + contagem_superiores}.\nCom {contagem_inferiores} valores abaixo do limite inferior e {contagem_superiores} valores acima do limite superior, e todos eles foram substituidos pelos seus respectivos limites em cada coluna de dados quantitativos. \n")

A quantidade de outliers detectados nas variáveis quantitativas foram 0.
Com 0 valores abaixo do limite inferior e 0 valores acima do limite superior, e todos eles foram substituidos pelos seus respectivos limites em cada coluna de dados quantitativos. 



# Data Analysis

In [16]:
# bimodal analysis

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import pearsonr
from datetime import datetime

sns.set_palette("deep") 

def analise_bimodal(df):
    """
    Analisa combinações bimodais entre colunas de um DataFrame e gera visualizações específicas.
    
    Parâmetros:
    df (pd.DataFrame): DataFrame com os dados a serem analisados
    """
    
    # Verificar tipos de cada coluna
    col_types = {}
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            col_types[col] = 'quantitativo'
        elif pd.api.types.is_datetime64_any_dtype(df[col]):
            col_types[col] = 'data'
        else:
            col_types[col] = 'qualitativo'
    
    # Percorrer todas as combinações únicas de colunas
    cols = df.columns.tolist()
    for i in range(len(cols)):
        for j in range(i + 1, len(cols)):
            col1 = cols[i]
            col2 = cols[j]
            type1 = col_types[col1]
            type2 = col_types[col2]
            
            print(f"\nAnalisando combinação: {col1} ({type1}) vs {col2} ({type2})")
            pdf_gen.add_text(f'''\n
            Analisando combinação: \n
            {col1} ({type1}) vs {col2} ({type2})\n
            '''.replace("\n", "<br />"))
            
            # Qualitativo vs Qualitativo
            if type1 == 'qualitativo' and type2 == 'qualitativo':
                analise_qual_qual(df, col1, col2)
            
            # Qualitativo vs Quantitativo
            elif (type1 == 'qualitativo' and type2 == 'quantitativo') or \
                 (type1 == 'quantitativo' and type2 == 'qualitativo'):
                if type1 == 'quantitativo':  # Garantir ordem qualitativo -> quantitativo
                    col1, col2 = col2, col1
                analise_qual_quant(df, col1, col2)
            
            # Quantitativo vs Quantitativo
            elif type1 == 'quantitativo' and type2 == 'quantitativo':
                analise_quant_quant(df, col1, col2)
            
            # Qualitativo vs Data
            elif (type1 == 'qualitativo' and type2 == 'data') or \
                 (type1 == 'data' and type2 == 'qualitativo'):
                if type1 == 'data':  # Garantir ordem qualitativo -> data
                    col1, col2 = col2, col1
                analise_qual_data(df, col1, col2)
            
            # Quantitativo vs Data
            elif (type1 == 'quantitativo' and type2 == 'data') or \
                 (type1 == 'data' and type2 == 'quantitativo'):
                if type1 == 'data':  # Garantir ordem quantitativo -> data
                    col1, col2 = col2, col1
                analise_quant_data(df, col1, col2)
            
            else:
                print(f"Combinação não tratada: {type1} vs {type2}")
                pdf_gen.add_text(f'''
                Combinação não tratada: \n
                {col1} ({type1}) vs {col2} ({type2})\n
                '''.replace("\n", "<br />"))

def analise_qual_qual(df, col1, col2):
    """Análise para duas variáveis qualitativas"""
    # Tabela de frequência
    freq = pd.crosstab(df[col1], df[col2], margins=True)
    freq_rel = pd.crosstab(df[col1], df[col2], normalize='all') * 100
    
    print("\nTabela de Frequência Absoluta:")
    display(freq)

    pdf_gen.add_dataframe(freq, style=False)

    pdf_gen.add_break()
    
    print("\nTabela de Frequência Relativa (%):")
    display(freq_rel.style.format("{:.2f}%"))

    pdf_gen.add_dataframe(freq_rel.style.format("{:.2f}%"), style=False)

    pdf_gen.add_break()
    
    # Gráfico de barras agrupadas
    fig = px.bar(df, x=col1, color=col2, barmode='group',
                 title=f"Distribuição de {col1} por {col2}")
    fig.show(renderer='iframe')

    fig.write_image("barplot.png", scale=2)

    pdf_gen.add_image("barplot.png")

def analise_qual_quant(df, col_qual, col_quant):
    """Análise para variável qualitativa vs quantitativa"""
    # Histogramas lado a lado
    fig = px.histogram(df, x=col_quant, color=col_qual, marginal="rug",
                       nbins=30, barmode='overlay', opacity=0.7,
                       title=f"Distribuição de {col_quant} por {col_qual}")
    fig.update_layout(bargap=0.1)
    fig.show(renderer='iframe')

    fig.write_image("histogram.png", scale=2)

    pdf_gen.add_image("histogram.png")
    
    # Boxplot para comparação
    fig = px.box(df, x=col_qual, y=col_quant, color=col_qual,
                 title=f"Distribuição de {col_quant} por {col_qual}")
    fig.show(renderer='iframe')

    fig.write_image("boxplot.png", scale=2)

    pdf_gen.add_image("boxplot.png")

def analise_quant_quant(df, col1, col2):
    """Análise para duas variáveis quantitativas"""
    # Scatter plot
    fig = px.scatter(df, x=col1, y=col2, trendline="ols",
                     title=f"Relação entre {col1} e {col2}")
    
    # Calcular correlação
    corr, p_valor = pearsonr(df[col1].dropna(), df[col2].dropna())
    
    # Remover linha de tendência se não houver correlação significativa
    if p_valor > 0.05:
        fig.data = [fig.data[0]]  # Mantém apenas o scatter
        fig.update_layout(annotations=[], title=f"Relação entre {col1} e {col2} (sem correlação significativa)")
    
    fig.add_annotation(text=f"Correlação: {corr:.2f} (p-valor: {p_valor:.3f})",
                       xref="paper", yref="paper",
                       x=0.05, y=0.95, showarrow=False)
    fig.show(renderer='iframe')

    fig.write_image("scatterlineplot.png", scale=2)

    pdf_gen.add_image("scatterlineplot.png")

def analise_qual_data(df, col_qual, col_data):
    """Análise para variável qualitativa vs data"""
    # Determinar granularidade temporal
    n_anos = df[col_data].dt.year.nunique()
    
    if n_anos >= 3:
        # Agrupar por ano
        df_temp = df.copy()
        df_temp['Ano'] = df_temp[col_data].dt.year
        df_grouped = df_temp.groupby(['Ano', col_qual]).size().reset_index(name='Contagem')
        
        fig = px.line(df_grouped, x='Ano', y='Contagem', color=col_qual,
                      title=f"Evolução Temporal de {col_qual} (por Ano)")
    else:
        # Agrupar por mês-ano
        df_temp = df.copy()
        df_temp['Mês-Ano'] = df_temp[col_data].dt.to_period('M').dt.to_timestamp()
        df_grouped = df_temp.groupby(['Mês-Ano', col_qual]).size().reset_index(name='Contagem')
        
        fig = px.line(df_grouped, x='Mês-Ano', y='Contagem', color=col_qual,
                      title=f"Evolução Temporal de {col_qual} (por Mês)")
    
    fig.update_traces(mode='lines+markers')
    fig.show(renderer='iframe')

    fig.write_image("lineplot.png", scale=2)

    pdf_gen.add_image("lineplot.png")

def analise_quant_data(df, col_quant, col_data):
    """Análise para variável quantitativa vs data"""
    # Determinar granularidade temporal
    n_anos = df[col_data].dt.year.nunique()
    
    if n_anos >= 3:
        # Agrupar por ano
        df_temp = df.copy()
        df_temp['Ano'] = df_temp[col_data].dt.year
        df_grouped = df_temp.groupby('Ano')[col_quant].mean().reset_index()
        
        fig = px.line(df_grouped, x='Ano', y=col_quant,
                      title=f"Série Temporal de {col_quant} (por Ano)")
    else:
        # Agrupar por mês-ano
        df_temp = df.copy()
        df_temp['Mês-Ano'] = df_temp[col_data].dt.to_period('M').dt.to_timestamp()
        df_grouped = df_temp.groupby('Mês-Ano')[col_quant].mean().reset_index()
        
        fig = px.line(df_grouped, x='Mês-Ano', y=col_quant,
                      title=f"Série Temporal de {col_quant} (por Mês)")
    
    # Adicionar média móvel
    fig.add_trace(go.Scatter(
        x=df_grouped.iloc[:, 0],
        y=df_grouped[col_quant].rolling(3, min_periods=1).mean(),
        name='Média Móvel (3 períodos)',
        line=dict(color='red', dash='dash')
    ))
    
    fig.update_traces(mode='lines+markers')
    fig.update_xaxes(rangeslider_visible=True)
    fig.show(renderer='iframe')

    fig.write_image("scatterplot.png", scale=2)

    pdf_gen.add_image("scatterplot.png")

In [17]:
# Heatmaps

def safe_pearson(x, y):
    """Calcula Pearson apenas para variáveis numéricas"""
    if pd.api.types.is_numeric_dtype(x) and pd.api.types.is_numeric_dtype(y):
        return pearsonr(x, y)[0]
    return np.nan

def safe_spearman(x, y):
    """Calcula Spearman para qualquer tipo após conversão"""
    try:
        return spearmanr(x, y).correlation
    except:
        return np.nan

def safe_mutual_info(x, y):
    """Calcula informação mútua de acordo com os tipos de dados"""
    try:
        if pd.api.types.is_numeric_dtype(y):
            return mutual_info_regression(x.values.reshape(-1, 1), y)[0]
        else:
            return mutual_info_classif(x.values.reshape(-1, 1), y)[0]
    except:
        return np.nan

def calculate_correlations(df):
    """Calcula todas as matrizes de correlação"""
    cols = df.columns
    n = len(cols)
    
    pearson = np.zeros((n, n))
    spearman = np.zeros((n, n))
    mi = np.zeros((n, n))
    
    # Pré-processamento para Spearman/MI
    df_encoded = df.copy()
    for col in df.select_dtypes(include=['object', 'category']):
        df_encoded[col] = LabelEncoder().fit_transform(df[col])
    
    for i in range(n):
        for j in range(n):
            pearson[i, j] = safe_pearson(df[cols[i]], df[cols[j]])
            spearman[i, j] = safe_spearman(df_encoded[cols[i]], df_encoded[cols[j]])
            mi[i, j] = safe_mutual_info(df_encoded[cols[i]], df_encoded[cols[j]])
    
    return {
        'Pearson': pd.DataFrame(pearson, index=cols, columns=cols),
        'Spearman': pd.DataFrame(spearman, index=cols, columns=cols),
        'Mutual_Info': pd.DataFrame(mi, index=cols, columns=cols)
    }

def plot_correlation_heatmaps(df):
    """Gera e exibe os heatmaps sem retornar valores"""
    correlations = calculate_correlations(df)
    
    # Heatmap Pearson
    fig_pearson = go.Figure(go.Heatmap(
        z=correlations['Pearson'],
        x=df.columns,
        y=df.columns,
        zmin=-1,
        zmax=1,
        colorscale='RdBu',
        text=np.round(correlations['Pearson'], 2),
        texttemplate="%{text}"
    ))
    fig_pearson.update_layout(title='Correlação Linear (Pearson)')
    fig_pearson.show(renderer='iframe')

    fig_pearson.write_image("heatmap_pearson.png", scale=2)

    pdf_gen.add_image("heatmap_pearson.png")
    
    
    # Heatmap Spearman
    fig_spearman = go.Figure(go.Heatmap(
        z=correlations['Spearman'],
        x=df.columns,
        y=df.columns,
        zmin=-1,
        zmax=1,
        colorscale='RdBu',
        text=np.round(correlations['Spearman'], 2),
        texttemplate="%{text}"
    ))
    fig_spearman.update_layout(title='Correlação Monotônica (Spearman)')
    fig_spearman.show(renderer='iframe')

    fig_spearman.write_image("heatmap_spearman.png", scale=2)

    pdf_gen.add_image("heatmap_spearman.png")
    
    
    # Heatmap Informação Mútua
    fig_mi = go.Figure(go.Heatmap(
        z=correlations['Mutual_Info'],
        x=df.columns,
        y=df.columns,
        colorscale='Viridis',
        text=np.round(correlations['Mutual_Info'], 2),
        texttemplate="%{text}"
    ))
    fig_mi.update_layout(title='Dependência Não-Linear (Informação Mútua)')
    fig_mi.show(renderer='iframe')

    fig_mi.write_image("heatmap_mi.png", scale=2)

    pdf_gen.add_image("heatmap_mi.png")
    

In [18]:
def analise_qualitativa(series):
    
    # unicos
    pdf_gen.add_text(f"""Há {len(series.unique())} valores unicos dos quais são: \n
    {series.unique()}
    \n""".replace("\n", "<br />"))
    
    # moda
    pdf_gen.add_text(f'''O que valor mais aparece (a moda) é \n
    {series.mode()}
    \n'''.replace("\n", "<br />"))

    
    # tabela de frequencia
    
    tabela_frequencia = pd.DataFrame(series.value_counts().reset_index())

    tabela_frequencia.columns = [series.name, 'Frequência Simples (ni)']

    tabela_frequencia['Frequência Relativa % (fi)'] = (tabela_frequencia['Frequência Simples (ni)'] / len(series)) * 100

    tabela_frequencia['Frequência Acumulada (Ni)'] = tabela_frequencia['Frequência Simples (ni)'].cumsum()

    tabela_frequencia['Frequência Relativa Acumulada % (Fi)'] = tabela_frequencia['Frequência Relativa % (fi)'].cumsum()
    
    tabela_frequencia_visual = tabela_frequencia.style.format({'Frequência Relativa % (fi)': '{:.2f}%', 'Frequência Relativa Acumulada % (Fi)': '{:.2f}%'})

    pdf_gen.add_dataframe(tabela_frequencia_visual, style=False)

    pdf_gen.add_break()

    
    # graficos

        # pizza
    fig_pie = px.pie(
        tabela_frequencia,
        names=series.name,
        values='Frequência Relativa % (fi)',
        title=f'Representação de cada {series.name}',
        color_discrete_sequence=px.colors.qualitative.Pastel
    )

        # melhorar a formatação
    fig_pie.update_traces(
        textposition='outside',
        textinfo='percent+label',
        pull=[0.12, 0.12, 0.12]  # Destacar as três primeiras fatias
    )
    
        # mostrar o gráfico
    fig_pie.show(renderer='iframe')
    
        # salvar como imagem
    fig_pie.write_image("grafico_pizza.png", scale=2)

    pdf_gen.add_image("grafico_pizza.png")

    # barras
    fig_bar = px.bar(
        tabela_frequencia,
        x=series.name,
        y='Frequência Relativa % (fi)',
        title=f'Representação de cada {series.name}',
        color=series.name,
        color_discrete_sequence=px.colors.qualitative.Set2
    )
    
        # personalizar o layout
    fig_bar.update_layout(
        xaxis_title=series.name,
        yaxis_title='Frequência Relativa % (fi)',
        showlegend=False,
        hovermode='x'
    )
    
        # adicionar valores nas barras
    fig_bar.update_traces(
        texttemplate='%{y}',
        textposition='outside'
    )
    
        # mostrar o gráfico
    fig_bar.show(renderer='iframe')
    
        # salvar como imagem
    fig_bar.write_image("grafico_barras.png", scale=2)

    pdf_gen.add_image("grafico_barras.png")

In [19]:
def analise_quantitativa(series):
    
    # media, moda, mediana
    media = series.mean()
    moda = series.median()
    mediana = series.mode()

    pdf_gen.add_text(f'''Os dados de {series.name} possuem:\n
    media: {media:.2f}\n
    moda: {moda:.2f}\n
    mediana: {mediana.iloc[0]:.2f}\n
    '''.replace("\n", "<br />"))


    # tabela de frequência intervalar
        
        # calculando número de classes (Regra de Sturges)
    n = len(series)
    k = int(1 + 3.322 * np.log10(n)) if n > 0 else 5
    if (k > 7):
        k = 7
    
        # calculando amplitude
    amplitude = series.max() - series.min()
    
        # calculando amplitude de cada classe
    h = amplitude / k

        # gerando os limites das classes
    limites_inferiores = np.linspace(series.min(), series.max() - h, k)
    limites_superiores = limites_inferiores + h
    
        # arredondando para melhor visualização
    limites_inferiores = np.round(limites_inferiores, 2)
    limites_superiores = np.round(limites_superiores, 2)

        # formata os intervalos de classe
    classes = [f"{li:.0f} ⊢ {ls:.0f}" for li, ls in zip(limites_inferiores, limites_superiores)]
    
        # calcula as frequências absolutas (ni)
    frequencias = []
    for li, ls in zip(limites_inferiores, limites_superiores):
        freq = ((series >= li) & (series < ls)).sum()
        frequencias.append(freq)
    
        # calcula os pontos médios (Xi)
    pontos_medios = (limites_inferiores + limites_superiores) / 2
    
        # cria o DataFrame base
    tabela = pd.DataFrame({
        series.name: classes,
        'Frequência (ni)': frequencias,
    })
    
    
        # calcula as demais colunas
    tabela['Frequência Relativa % (fi)'] = np.round(tabela['Frequência (ni)'] / n, 4)
    tabela['Frequência Relativa % (fi)'] = (tabela['Frequência Relativa % (fi)'] * 100).round(2)
    tabela['Frequência Acumulada (Ni)'] = tabela['Frequência (ni)'].cumsum()
    tabela['Frequência Relativa Acumulada % (Fi)'] = np.round(tabela['Frequência Acumulada (Ni)'] / n, 4)
    tabela['Frequência Relativa Acumulada % (Fi)'] = (tabela['Frequência Relativa Acumulada % (Fi)'] * 100).round(2)
    tabela['Ponto Médio da Classe (Xi)'] = np.round(pontos_medios, 2)
    
        # adiciona a porcentagem para facilitar leitura
    tabela_visual = tabela.style.format({'Frequência Relativa % (fi)': '{:.2f}%', 'Frequência Relativa Acumulada % (Fi)': '{:.2f}%', 'Ponto Médio da Classe (Xi)': '{:.2f}'})
    
    pdf_gen.add_dataframe(tabela_visual, style=False)

    pdf_gen.add_break()

    
    # dispersão
    # desvio padrão e coeficiente de variação
    desvio = series.std()
        
    pdf_gen.add_text(f"Desvio padrão: \n{desvio:.4f} \n\nCoeficiente de variação: \n{(desvio/media)*100:.2f}%\n\n".replace("\n", "<br />"))
        
    # interpretação
    if desvio/media > 0.3:
        pdf_gen.add_text("Os dados estão relativamente dispersos em relação à média\n".replace("\n", "<br />"))
    else:
        pdf_gen.add_text("Os dados estão relativamente concentrados em torno da média\n".replace("\n", "<br />"))


    
    # boxplot completo com estatísticas
    fig = go.Figure()
    
    fig.add_trace(go.Box(
        y=series,
        name='Distribuição',
        boxpoints='outliers',  # Mostra outliers individuais
        marker_color='#1f77b4',
        line_color='#1f77b4',
        fillcolor='lightblue',
        jitter=0.5,  # Espaçamento dos pontos
        whiskerwidth=0.2,
        hoverinfo='y+name'
    ))
    
    # Adicionando estatísticas como anotações
    stats = series.describe()
    annotations = [
        dict(
            x=0.05,
            y=stats['mean'],
            xref='paper',
            yref='y',
            text=f'Média: {stats["mean"]:.2f}',
            showarrow=True,
            arrowhead=1,
            ax=-50
        ),
        dict(
            x=0.05,
            y=stats['50%'],
            xref='paper',
            yref='y',
            text=f'Mediana: {stats["50%"]:.2f}',
            showarrow=True,
            arrowhead=1,
            ax=-50
        )
    ]
    
    fig.update_layout(
        title=f'Boxplot of {series.name}',
        yaxis_title='Valores',
        showlegend=False,
        annotations=annotations,
        hovermode='y unified',
        height=600
    )
    
    fig.show(renderer='iframe')

    fig.write_image("boxplot.png", scale=2)

    pdf_gen.add_image("boxplot.png")



    # simetria/assimetria

        # configuração do tema
    pio.templates.default = "plotly_white"
    
        # entrada dos dados 
    dados = series

    media = float(dados.mean())  # Convertendo para float nativo
    mediana = float(dados.median())
    modas = dados.mode().values  # Array numpy com as modas

    title = f"Histogram of {series.name}"

    valores = dados

     # Tentar adicionar KDE
    fig = go.Figure()
    
    # Adicionar histograma
    fig.add_trace(go.Histogram(
        x=valores,
        name='Histograma',
        nbinsx=30,
        marker_color='#1f77b4',
        opacity=0.7,
        histnorm='probability density',
        hovertemplate='Intervalo: %{x:.2f}<br>Frequência: %{y:.4f}<extra></extra>'
    ))
    
    # Adicionar curva KDE (se possível)
    try:
        if len(np.unique(valores)) > 1:  # Verifica se há variância
            kde = gaussian_kde(valores)
            x_kde = np.linspace(np.min(valores), np.max(valores), 500)
            y_kde = kde(x_kde)
            
            fig.add_trace(go.Scatter(
                x=x_kde, y=y_kde,
                name='Densidade KDE',
                line=dict(color='#ff7f0e', width=2),
                hovertemplate='Valor: %{x:.2f}<br>Densidade: %{y:.4f}<extra></extra>'
            ))
        else:
            pdf_gen.add_text(f"Os dados de {series.name} são constantes\n".replace("\n", "<br />"))
            raise ValueError("Dados constantes")
            
    except Exception as e:
        pdf_gen.add_text(f"O KDE de {series.name} não pôde ser calculado.\nMotivo:\n{str(e)}\n".replace("\n", "<br />"))
        print(f"Aviso: Não foi possível calcular KDE - {str(e)}")
        
    
    # Posições verticais para as anotações (evitar sobreposição)
    y_positions = {
        'media': 0.9,
        'mediana': 0.7,
        'moda': 0.5
    }
    
    # Adicionar linha da média
    fig.add_vline(
        x=media,
        line=dict(color='red', dash='dash', width=2),
        annotation_text=f'Média: {media:.2f}',
        annotation_position="top right",
        annotation_y=y_positions['media'],
        annotation_yanchor="top",
        annotation_font=dict(color='red')
    )
    
    # Adicionar linha da mediana
    fig.add_vline(
        x=mediana,
        line=dict(color='green', width=2),
        annotation_text=f'Mediana: {mediana:.2f}',
        annotation_position="bottom right",
        annotation_y=y_positions['mediana'],
        annotation_yanchor="top",
        annotation_font=dict(color='green')
    )
    
    # Adicionar linhas para moda(s)
    for i, moda in enumerate(modas):
        fig.add_vline(
            x=moda,
            line=dict(color='purple', dash='dot', width=2),
            annotation_text=f'Moda {i+1}: {moda:.2f}' if len(modas) > 1 else f'Moda: {moda:.2f}',
            annotation_position="top left",
            annotation_y=y_positions['moda'] - (i*0.1),  # Deslocamento para múltiplas modas
            annotation_yanchor="top",
            annotation_font=dict(color='purple')
        )
    
    # Layout final
    fig.update_layout(
        title=title,
        xaxis_title='Valores',
        yaxis_title='Densidade',
        hovermode='x unified',
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        height=600,
        margin=dict(t=50, b=50)
    )
    
    fig.show(renderer='iframe')

    fig.write_image("histograma.png", scale=2)      
        
    pdf_gen.add_image("histograma.png")       
        

## Analysis functions

In [20]:
for coluna in df.columns:
    if (df[coluna].dtype == 'string'):
        pdf_gen.add_header(f'''\n{' ANALISE QUALITATIVA '} \n'''.replace("\n", "<br />"))
        analise_qualitativa(df[coluna])

    elif(df[coluna].dtype == ('float' or 'int')):
        pdf_gen.add_header(f'''\n{' ANALISE QUANTITATIVA '} \n'''.replace("\n", "<br />"))
        analise_quantitativa(df[coluna])


pdf_gen.add_header(f'''\n{' CORRELAÇÕES COM MAPAS DE CALOR '} \n'''.replace("\n", "<br />"))
plot_correlation_heatmaps(df)


pdf_gen.add_header(f'''\n{' ANÁLISES BIMODAIS '} \n'''.replace("\n", "<br />"))
analise_bimodal(df.iloc[:, 1:])

Aviso: Não foi possível calcular KDE - Dados constantes


Aviso: Não foi possível calcular KDE - Dados constantes



Analisando combinação: Gender (qualitativo) vs Age (quantitativo)



Analisando combinação: Gender (qualitativo) vs City (qualitativo)

Tabela de Frequência Absoluta:


City,'Less Delhi','Less than 5 Kalyan',3.0,Agra,Ahmedabad,Bangalore,Bhavna,Bhopal,Chennai,City,...,Saanvi,Srinagar,Surat,Thane,Vaanya,Vadodara,Varanasi,Vasai-Virar,Visakhapatnam,All
Gender,,,,,,,,,,,,,,,,,,,,,
Female,1,0,1,446,404,380,0,421,494,2,...,0,661,491,527,0,399,291,718,486,12354
Male,0,1,0,648,547,387,2,513,391,0,...,2,711,587,612,1,295,394,572,483,15547
All,1,1,1,1094,951,767,2,934,885,2,...,2,1372,1078,1139,1,694,685,1290,969,27901


...

Family History of Mental Illness,No,Yes,All
Financial Stress,,,
1.0,2577,2544,5121
2.0,2772,2289,5061
3.0,2714,2512,5226
4.0,2899,2876,5775
5.0,3435,3280,6715
?,1,2,3
All,14398,13503,27901



Tabela de Frequência Relativa (%):


Family History of Mental Illness,No,Yes
Financial Stress,,
1.0,9.24%,9.12%
2.0,9.94%,8.20%
3.0,9.73%,9.00%
4.0,10.39%,10.31%
5.0,12.31%,11.76%
?,0.00%,0.01%



Analisando combinação: Financial Stress (qualitativo) vs Depression (quantitativo)



Analisando combinação: Family History of Mental Illness (qualitativo) vs Depression (quantitativo)


In [21]:
pdf_file = pdf_gen.generate_pdf("analise_descritiva.pdf")
print(f"PDF gerado com sucesso: {pdf_file}")

# Opcional: fazer download no Kaggle
from IPython.display import FileLink
FileLink(pdf_file)

PDF gerado com sucesso: analise_descritiva.pdf


/kaggle/working/analise_descritiva.pdf